In [1]:
import pandas as pd
import eform_query_processor as eqp
import os

In [2]:
# setting up the absolute savepath
save_path = r'C:\Users\aange\OneDrive\Desktop\Personal Python Projects\ferc_data_project\ferc_714_xml_files_test'

In [3]:
# loading in df containing files we want to grab
source_df = pd.read_csv('process_check/eform_filtered_data_2024-09-14.csv')

source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID        153 non-null    object
 1   Company    153 non-null    object
 2   Form       153 non-null    object
 3   Year       153 non-null    int64 
 4   Period     153 non-null    object
 5   Date/Time  153 non-null    object
 6   Status     153 non-null    object
 7   Filing ID  153 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 9.7+ KB


In [4]:
# checking the first 5
source_df_f5 = source_df[:5]

print(source_df_f5.info())

print(source_df_f5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID        5 non-null      object
 1   Company    5 non-null      object
 2   Form       5 non-null      object
 3   Year       5 non-null      int64 
 4   Period     5 non-null      object
 5   Date/Time  5 non-null      object
 6   Status     5 non-null      object
 7   Filing ID  5 non-null      int64 
dtypes: int64(2), object(6)
memory usage: 452.0+ bytes
None
       CID                                            Company      Form  Year  \
0  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
1  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
2  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
3  C004245  Salt River Project Agricultural Improvement an...  Form 714  2023   
4  C000771                         Southwest 

In [5]:
# create a list of filing id
filing_ids = list(source_df_f5['Filing ID'])
print(filing_ids)

[192310, 192309, 192306, 192277, 182167]


In [6]:
# return files
eqp.return_eform_filings(filing_ids=filing_ids, 
                         save_path=save_path)

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Southwest Power Pool, Inc._ferc_714_Q42023_192310.xml
Southwest Power Pool, Inc._ferc_714_Q42023_192309.xml
Southwest Power Pool, Inc._ferc_714_Q42023_192306.xml
Salt River Project Agricultural Improvement and Power District_ferc_714_Q42023_192277.xml
Southwest Power Pool, Inc._ferc_714_Q42023_182167.xml


Now we want to get a sense of how to sort our directories by time stamp
so that we can return a list of file paths to match with out pre existing
filing ids dataframe, allowing us to rename duplicate named files to assess
which files may have been revises or simply reuploaded at a later time.

In [13]:
# function to sort directory provided by either name or time
def sort_directory(sort_by: str, dir_path: str):

    '''
    In essence, first select whether you want to sort directory by name or by date created
    '''

    # purpose is to return the time the file was created
    def get_creation_time(item):
        item_path = os.path.join(dir_path, item)
        return os.path.getctime(item_path)
    
    # items meaning the files wihthin the directory path defined
    items = os.listdir(dir_path)

    if sort_by == 'date':
        '''
        in this case, user would like to sort directory path files by date
        '''
        print('Sorting files in path by creation date!')

        # sorting our items from oldest to newest
        sorted_items = sorted(items, key=get_creation_time)

        return sorted_items

    elif sort_by == 'name':
        '''
        in this case we would like to sort directory path files by name
        '''
        print('Sorting files in path by name!')

    else: 
        print('Please redefine how you would like to sort!')


In [14]:
# folder_path to test
fpath_714 = 'ferc_714_xml_files_test'

# attempt to check function
test_case = sort_directory(sort_by='date', dir_path=fpath_714)
print(test_case)


Sorting files in path by creation date!
['spp-20231231.xbrl', 'spp-20231231 (1).xbrl', 'spp-20231231 (2).xbrl', 'wk-20231231.xml', 'spp-20231231 (3).xbrl']


In [34]:
# checking how to combines values of row data
date_times = list(source_df_f5['Date/Time'])
company_names = list(source_df_f5['Company'])

for date in date_times:
    # grabbing index value of 'date' to reference it in company names lis
    print(f'{company_names[date_times.index(str(date))]}_{date}')




Southwest Power Pool, Inc._7/9/2024, 4:25 PM
Southwest Power Pool, Inc._7/9/2024, 1:58 PM
Southwest Power Pool, Inc._7/8/2024, 9:13 AM
Salt River Project Agricultural Improvement and Power District_7/3/2024, 12:24 PM
Southwest Power Pool, Inc._6/17/2024, 8:45 AM


In [ ]:
# okay, now we want to match the provided file list with the pandas df

def rename_all_files(xml_df: str, sorted_dpath: str):

    '''
    now we will first iterate through xml df, grab rows of data to set up a unique file name
    and then rename the file to that name
    '''
    